In [1]:
# https://the-algorithms.com/algorithm/karatsuba

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


## karatsuba Algorithm

In [2]:
def generate_random_numbers(num_digits):
    # Gerar o limite superior e inferior com base no número de dígitos
    lower_bound = 10 ** (num_digits - 1)
    upper_bound = (10 ** num_digits) - 1
    
    # Gerar dois números aleatórios dentro do intervalo especificado
    random_number1 = random.randint(lower_bound, upper_bound)
    random_number2 = random.randint(lower_bound, upper_bound)
    
    return random_number1, random_number2

def karatsuba(a: int, b: int) -> int:
    """
    >>> karatsuba(15463, 23489) == 15463 * 23489
    True
    >>> karatsuba(3, 9) == 3 * 9
    True
    """
    if len(str(a)) == 1 or len(str(b)) == 1:
        return a * b

    m1 = max(len(str(a)), len(str(b)))
    m2 = m1 // 2

    a1, a2 = divmod(a, 10**m2)
    b1, b2 = divmod(b, 10**m2)

    x = karatsuba(a2, b2)
    y = karatsuba((a1 + a2), (b1 + b2))
    z = karatsuba(a1, b1)

    return (z * 10 ** (2 * m2)) + ((y - z - x) * 10 ** (m2)) + (x)

## Frequency Count Method

In [3]:
X_y = []
i=10
for n in range(2,i+1):
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(karatsuba)
  
  # create numbers with length n
  num1, num2 = generate_random_numbers(n)
  lp_wrapper(num1, num2)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [4]:
#resultados_com_menor_loss = []
repeat = 4
registros = []
unary_operators_list = ["log", "square", "cube", "sqrt", "round", "exp", "abs"]

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    reg1 = PySRRegressor(
      #binary_operators=["*", "+"],
      unary_operators=unary_operators_list
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)
    
sys.stdout = original_stdout



[ Info: Started!
0.0%┣                                             ┫ 0/600 [00:00<-2:-13, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           4.275e+04  1.594e+01  y = x₀                                        2           2.579e+04  5.054e-01  y = square(x₀)                                4           8.782e+02  1.690e+00  y = square(x₀ + x₀)                           6           8.670e+02  6.417e-03  y = square(x₀ + (x₀ - 0.12824))               8           8.616e+02  3.164e-03  y = (square((x₀ + x₀) + 0.078969) - 5.9031)   -----------------------

In [5]:
registros_ = registros
file_pickle = 'dados_v2.pck'

In [6]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [7]:
def salvar_dados(dados, key, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)
  else:
    dados_exist = {}
    
  valor_original = dados_exist.get(key)
  if valor_original == None:
    dados_exist.update({key: [dados]})
  else:
    if isinstance(valor_original, list):
      valor_original.append(dados)
    else:
      dados_exist.update({key: [dados]})

  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)

caminho_arquivo = file_pickle
novos_dados = lista_melhor_valor[0:3] + [lista_melhor_valor[4]]

salvar_dados(novos_dados, 'karatsuba_N^1.585', caminho_arquivo)

In [8]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = file_pickle
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items():
  print('\u25CF', k)
  for index, item in enumerate(v):
    if index == len(v)-1:
       print('└─', item)
    else:
      print('├─', item)
  print('==========================')

Conteúdo do arquivo pickle:
● binary_search_logx
├─ [5, 1.5291587, 2.194796893650787, log(x0)**2 + 13.966388]
├─ [5, 1.2686663, 2.221314226989192, 10.953988*x0**(1/4)]
└─ [5, 1.2686664, 2.221314062669807, 10.9540106810246*x0**(1/4)]
● karatsuba_N^1.585
└─ [8, 506.257, 0.32556235249161525, 2.8521053871194*x0**2 + sqrt(exp(x0))]
